# Connect to Racebox Mini and read raw accelerometry trace

### The Tech Stack

BLE(bluetooth low energy) is a communication standard between computers and bluetooth devices. 

[Bleak](https://github.com/hbldh/bleak) is a platform-agnostic python package that connects to BLE devices from a computer. It also implements handy functions to read data

### Scan for available devices

In [1]:
import asyncio
from bleak import BleakScanner

async def main():
    devices = await BleakScanner.discover()
    for d in devices:
        print(d)

# for Python script
# asyncio.run(main())

# from within Jupyter
# https://stackoverflow.com/questions/55409641/asyncio-run-cannot-be-called-from-a-running-event-loop-when-using-jupyter-no
await main()

70:7A:60:A5:AD:F1: None
19:48:0D:F6:9F:6E: None
04:3E:D6:11:16:3B: None
20:3A:AB:0B:7B:FD: None
88:C6:26:CF:4B:3B: None
D4:4D:A4:2A:80:D5: Venus_D44DA42A80D5
0A:C5:8C:68:E5:FF: None
D4:AD:FC:52:F9:37: ihoment_H7141_F937
16:10:E4:A0:62:3D: None
4E:C6:04:7A:CA:6C: None
A4:E5:7C:6C:3F:AA: ScentAir
68:0E:22:EC:B7:4D: None
3A:54:F3:43:26:B8: None
71:A9:65:B5:58:B5: None
6D:C7:FA:CA:E8:C6: None
C9:CD:EB:7E:03:81: None
F4:4D:57:51:28:B9: None
E1:DE:52:46:5A:BF: None
7B:4C:F5:61:24:E0: None
72:F8:63:53:26:6B: None
62:5A:39:08:E7:57: None
1F:EB:64:0D:86:0F: None
5A:F0:08:B7:C1:F9: None
E0:1C:DB:30:66:E1: None
F1:DD:03:00:93:05: None
C1:9E:EB:FB:6F:19: None
61:C0:90:91:AD:B5: None
5A:5C:00:36:8C:00: None
1F:2C:8E:AC:D3:B2: None
6A:54:65:FF:4A:80: None
D3:3F:F0:45:5A:75: RaceBox Mini 1221405970
72:95:87:E8:12:9F: None
53:C7:D5:91:BC:16: LE-Night
FC:3B:FE:E3:4B:28: None
2A:98:40:E0:73:4D: None
4B:EC:C8:F6:8A:1D: None
DB:6D:00:86:48:B0: None
7E:EC:DD:AE:1F:65: None
F2:BF:4A:03:20:19: None


### Connect to RaceBox Mini

In [2]:
from bleak import BleakClient, BleakScanner
from bleak.exc import BleakError

ADDRESS = ("D3:3F:F0:45:5A:75")

async def main(ble_address: str):
    device = await BleakScanner.find_device_by_address(ble_address, timeout=20.0)
    if not device:
        raise BleakError(f"A device with address {ble_address} could not be found.")
    async with BleakClient(device) as client:
        print("Services:")
        for service in client.services:
            print(service)
            
await main(ADDRESS)

# print list of available services

Services:
00001800-0000-1000-8000-00805f9b34fb (Handle: 1): Generic Access Profile
00001801-0000-1000-8000-00805f9b34fb (Handle: 10): Generic Attribute Profile
6e400001-b5a3-f393-e0a9-e50e24dcca9e (Handle: 11): Nordic UART Service
0000180a-0000-1000-8000-00805f9b34fb (Handle: 17): Device Information


TimeoutError: 

### Subscribe for messages

In [ ]:
import asyncio
import sys
from itertools import count, takewhile
from typing import Iterator

from bleak import BleakClient, BleakScanner
from bleak.backends.characteristic import BleakGATTCharacteristic
from bleak.backends.device import BLEDevice
from bleak.backends.scanner import AdvertisementData

# https://www.racebox.pro/products/racebox-mini/protocol-documentation?k=63eec19e3cd15f49780062e1
UART_SERVICE_UUID = "6E400001-B5A3-F393-E0A9-E50E24DCCA9E"
UART_RX_CHAR_UUID = "6E400002-B5A3-F393-E0A9-E50E24DCCA9E"
UART_TX_CHAR_UUID = "6E400003-B5A3-F393-E0A9-E50E24DCCA9E"